# Рекомендация тарифов

Задача: построить модель для задачи классификации, которая выберет подходящий тариф. Минимальное значение *accuracy* на тестовой выборке: 0.75

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from joblib import dump
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/Users/vladamalkina/Desktop/ЯП проекты/проект 6/users_behavior_sixth_project.csv')
display(data.head())

,Unnamed: 0,calls,minutes,messages,mb_used,is_ultra
0,0,40.0,311.90,83.0,19915.42,0
1,1,85.0,516.75,56.0,22696.96,0
2,2,77.0,467.66,86.0,21060.45,0
3,3,106.0,745.53,81.0,8437.39,1
4,4,66.0,418.74,1.0,14502.75,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3214 non-null   int64  
 1   calls       3214 non-null   float64
 2   minutes     3214 non-null   float64
 3   messages    3214 non-null   float64
 4   mb_used     3214 non-null   float64
 5   is_ultra    3214 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 150.8 KB


## Разделим данные на выборки

In [4]:
data_train, data_common = train_test_split(data,  test_size = 0.4, random_state = 12345)
data_valid, data_test = train_test_split(data_common,  test_size = 0.5, random_state = 12345)

Разбили данные на три выборки: обучающую, валидационную и тестовую, соотношение 3:1:1 (60% : 20% : 20%)

## Исследование моделей

In [5]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra'] 
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

features - признаки
target - целевой признак

In [6]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1928, 5)
(1928,)
(643, 5)
(643,)
(643, 5)
(643,)


Дерево решений:

In [7]:
best_result = 0
best_model_decision_tree = None
for depth in range(1, 6):
    model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model_decision_tree.fit(features_train, target_train)
    predictions_valid =  model_decision_tree.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model_decision_tree = model_decision_tree
        best_result = result 

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
#criterion='gini'

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


Случайный лес:

In [8]:
best_model_forest = None
best_result = 0

for est in range(1, 11):
    for depth in range (1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train) 
        result = model_forest.score(features_valid, target_valid) 
        if result > best_result:
            best_model_forest = model_forest
            best_result = result 

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247


Логистическая регрессия:

In [9]:
model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model_regression.fit(features_train, target_train)
result = model_regression.score(features_valid, target_valid) 
dump(model_regression, 'model_9_1.joblib')
print("Accuracy наилучшей модели на валидационной выборке:", result)

Accuracy наилучшей модели на валидационной выборке: 0.7122861586314152


Самое высокое значение accuracy получили с помощью модели случайный лес: 0.80248833592535.

Самое низкое значение accuracy получили с помощью модели логистическая регрессия: 0.7107309486780715.

Значение accuracy модели дерево решений: 0.7853810264385692.

## Проверка модели на тестовой выборке

Дерево решений:

In [10]:
test_predictions = model_decision_tree.predict(features_test)
print("Тестовая выборка:", accuracy_score(target_test, test_predictions))

Тестовая выборка: 0.7900466562986003


Случайный лес:

In [11]:
test_predictions = best_model_forest.predict(features_test)
print("Тестовая выборка:", accuracy_score(target_test, test_predictions))

Тестовая выборка: 0.8055987558320373


Логистическая регрессия:

In [12]:
test_predictions = model_regression.predict(features_test)
print("Тестовая выборка:", accuracy_score(target_test, test_predictions))

Тестовая выборка: 0.6842923794712286


Самое высокое значение accuracy получили с помощью модели случайный лес: 0.7962674961119751.

Самое низкое значение accuracy получили с помощью модели логистическая регрессия: 0.6842923794712286.

Значение accuracy модели дерево решений: 0.7838258164852255.

## Проверка модели на адекватность

In [13]:
model_check = DummyClassifier(strategy = 'most_frequent', random_state = 12345)
model_check.fit(features_train, target_train)
print('Accuracy', model_check.score(features_test, target_test))

Accuracy 0.6842923794712286
